In [ ]:
from sportsreference.nfl.teams import Teams
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

teams = Teams(year= '2020')
teams_df = teams.dataframes
teams_df.sort_values(by=['name'], inplace=True, ascending=True)
print(teams_df['name'])
teams_df.set_index('name', inplace=True)
teams_df.drop(['first_downs', 'first_downs_from_penalties',  'games_played','losses', 'abbreviation','pass_attempts', 'pass_completions', 'pass_first_downs','plays', 'points_contributed_by_offense','post_season_result', 'rush_attempts', 'rush_first_downs', 'wins'], axis=1, inplace= True)
#stats.zscore(teams_df, axis =1, nan_policy="omit")
#print(list(teams_df.columns))
for (columnName, columnData) in teams_df.iteritems(): 
    if columnName != 'name':
        teams_df[columnName] = stats.zscore(columnData)
teams_df['fumbles'] *= -1
teams_df['interceptions'] *= -1
teams_df['penalties'] *= -1
teams_df['percent_drives_with_turnovers'] *= -1
teams_df['points_against'] *= -1
teams_df['turnovers'] *= -1
teams_df['yards_from_penalties'] *= -1
#print(teams_df.head())
#print(list(teams_df.columns))

In [ ]:
rank = pd.Series()
rank['defensive_simple_rating_system'] = 5
rank['fumbles'] =0
rank['interceptions'] =0
rank['margin_of_victory'] = 3
rank['offensive_simple_rating_system'] = 5
rank['pass_net_yards_per_attempt'] = .5
rank['pass_touchdowns'] = 1
rank['pass_yards'] =1
rank['penalties'] =1
rank['percent_drives_with_points'] =2
rank['percent_drives_with_turnovers'] = 2
rank['points_against'] =1
rank['rank'] = 0
rank['rush_touchdowns'] = 1
rank['rush_yards'] = 3
rank['rush_yards_per_attempt'] =1
rank['simple_rating_system'] = 7
rank['strength_of_schedule'] = 4
rank['turnovers'] = 3
rank['win_percentage'] = 6
rank['yards'] = 2
rank['yards_from_penalties'] = .5
rank['yards_per_play'] = 2  
sum = rank.sum() 
rank/=sum

In [ ]:
for (columnName, columnData) in rank.iteritems(): 
    teams_df[columnName]*= columnData
    print(rank[columnName])

In [ ]:
teams_df['sum'] = 0.0
for i, row in teams_df.iterrows():
    teams_df.at[i, 'sum'] = row['defensive_simple_rating_system':].sum()
teams_df.sort_values(by=['sum'], inplace=True, ascending=False)
print(teams_df['sum'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.kdeplot(teams_df['sum'])



In [ ]:
teams_df['zscores'] = stats.zscore(teams_df['sum'])
teams_df['percentile'] =  1- stats.norm.sf(teams_df['zscores'])
print(teams_df['percentile'])

In [ ]:
def predict (team1, team2):
    t1 = teams_df['percentile'].loc[team1]
    t2 = teams_df['percentile'].loc[team2]
    #return (teams_df['percentile'].loc[team1] - teams_df['percentile'].loc[team2])/2 + .5
    p = 1/(10**(-(t1 - t2))+1)
    return probToMoneyLine(p)

In [ ]:
def probToMoneyLine (prob):
    ml = 0
    prob*=100
    if prob >50:
        ml = -(prob/(100 - prob)) * 100
    elif prob < 50:
        ml = (((100 - prob)/prob) * 100)
    else:
        ml = 100
    return ml

In [ ]:
predict('Tennessee Titans', 'Las Vegas Raiders')

In [ ]:
import json
import requests
import argparse

parser = argparse.ArgumentParser(description='Sample')
parser.add_argument('--api-key', type=str, default='')
args, unknown = parser.parse_known_args()


API_KEY = '58f860df380e5b01f108f9418584b714'

SPORT = 'americanfootball_nfl' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGION = 'us' # uk | us | eu | au

MARKET = 'h2h' # h2h | spreads | totals

ODDSFORMAT  = 'american'



# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
    'api_key': API_KEY,
    'sport': SPORT,
    'region': REGION,
    'mkt': MARKET,
    'oddsFormat': ODDSFORMAT,
})

odds_json = json.loads(odds_response.text)
games = []
if not odds_json['success']:
    print(odds_json['msg'])
else:
    print('Number of events:', len(odds_json['data']))
    print(odds_json['data'][0]['commence_time'])
    first = odds_json['data']
    for i, game in enumerate(odds_json['data'], start=0):
        games.append({})
        games[i]['teams'] = game['teams']
        games[i]['home'] = game['home_team']

        for site in game['sites']:
            if site['site_nice'] == 'Caesars':
                games[i]['odds'] = site['odds']['h2h']

    # Check your usage
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

In [ ]:
print(games)

In [ ]:
for game in games:
    team1 = game['teams'][0]
    team2 = game['teams'][1]
    print(team1, ' vs', team2)
    print('Predicted Line for', team1,'is', predict(team1, team2))
    print('Actual Line for', team1,'is', game['odds'][0] )
